<a href="https://colab.research.google.com/github/htapiagroup/sistemas-distribuidos-evaluacion-intersemestral-ginnaRenz/blob/master/SistDist_Examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Equipo:
Ismael López Martínez <br/>
Ginna Monzerrat Nava Enríquez<br/>
Francisco García Ortega<br/>
José David Cobos Trujillo<br/>
Examen: https://colab.research.google.com/drive/13lToRPMX3G1jm6Owj195lWWQQ5OLEhiB

In [0]:
!wget https://archive.org/download/nycTaxiTripData2013/trip_data.7z #descarga del dataset

--2019-06-24 04:02:22--  https://archive.org/download/nycTaxiTripData2013/trip_data.7z
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia802501.us.archive.org/1/items/nycTaxiTripData2013/trip_data.7z [following]
--2019-06-24 04:02:23--  https://ia802501.us.archive.org/1/items/nycTaxiTripData2013/trip_data.7z
Resolving ia802501.us.archive.org (ia802501.us.archive.org)... 207.241.228.191
Connecting to ia802501.us.archive.org (ia802501.us.archive.org)|207.241.228.191|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4102781969 (3.8G) [application/x-7z-compressed]
Saving to: ‘trip_data.7z’

trip_data.7z        100%[===================>]   3.82G  11.6MB/s    in 5m 45s  

2019-06-24 04:08:09 (11.3 MB/s) - ‘trip_data.7z’ saved [4102781969/4102781969]



In [2]:
ls

sample_data/  trip_data.7z


In [3]:
!7z l trip_data.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 4102781969 bytes (3913 MiB)

Listing archive: trip_data.7z

--
Path = trip_data.7z
Type = 7z
Physical Size = 4102781969
Headers Size = 433
Method = LZMA:24
Solid = -
Blocks = 12

   Date      Time    Attr         Size   Compressed  Name
------------------- ----- ------------ ------------  ------------------------
2014-05-12 22:02:53 ....A   2459600863    345687208  trip_data_1.csv
2014-01-15 06:55:32 ....A   2502278613    358624764  trip_data_10.csv
2014-01-15 07:34:24 ....A   2395449056    342987617  trip_data_11.csv
2014-01-15 08:09:28 ....A   2327237611    331056148  trip_data_12.csv
2014-05-12 22:08:09 ....A   2328673265    327374725  trip_data_2.csv
2013-08-26 02:19:46 ....A   2622301287    371026139  trip_data

In [0]:
import pandas as pd
import numpy as np
import psutil, os 
import dask.dataframe as dd
import gc
import time
import datetime as dt

In [0]:
#Plantilla para descomprimir los archivos del archivo 7z
plantilla = 'trip_data_{:01d}.csv'

In [0]:
#Arreglo contiene el nombre de los archivos que eventualmente se descomprimiran del original
files =[ plantilla.format(month) for month in range(1,13)]

##1 y 2 Procesamiento de archivo trip_data.7z
Nuestro procedimiento consistio en descomprimir archivo por archivo, procesarlo, finalmente borrarlo.<br/>
Con el objetivo de no llenar el espacio en DD

In [0]:
def leer_archivo_depurado(fileName):  
  plantilla7z='7z x trip_data.7z  -o/content '+ fileName
  #print (plantilla7z)
  os.system(plantilla7z)
  data=dd.read_csv(file,low_memory=False, parse_dates=True, usecols=[0,5,6,7,8,9,10,11,12,13], 
                   names=['medallion','pickup_datetime','dropoff_datetime','passenger_count',
                          'trip_time_in_secs','trip_distance','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],header=0, dtype={'trip_distance': 'float64','trip_time_in_secs': 'int64', 'medallion': 'str', 'passenger_count': 'int64','pickup_longitude': 'float64','pickup_latitude':'float64','dropoff_longitude': 'float64', 'dropoff_latitude':'float64'}).compute() # se carga a dataset de pandas    
  os.system('rm '+fileName)  
  return data

In [0]:
def leer_archivo_puros(fileName):
  plantilla7z='7z x trip_data.7z  -o/content '+ fileName
  #print (plantilla7z)
  os.system(plantilla7z)
  data=dd.read_csv(file,low_memory=False, parse_dates=True, dtype={' store_and_fwd_flag':'object'}).compute() # se carga a dataset    
  os.system('rm '+fileName)  
  return data
  

##3.- Responder las siguientes preguntas:<br/>
A) Cuantas columnas contiene cada archivo de datos descomprimido.<br/>
B) Cuantos renglones tiene cada archivo.

In [0]:
dfContar = pd.DataFrame(columns=['Name', 'Columns', 'Rows'])


In [0]:
#Con base a la plantilla de los 12 archivos
#Se procesaron de forma secuencial uno tras otro con la librerias dask
#Para intercalar el uso de los recursos (Memoria, Disco Duro)
#Finalmente el resultado lo dejamos en un DataFrame de Pandas pues no implica gran uso de de los recursos
for file in files:
  df=leer_archivo_puros(file)
  total_rows=len(df.axes[0])
  total_cols=len(df.axes[1])
  new_row=pd.DataFrame({'Name' : file , 'Columns' : total_cols, 'Rows':total_rows},index=[0])  
  dfContar=pd.concat([new_row,dfContar]).reset_index(drop=True)

In [13]:
  dfContar.sort_values(by=['Name'])
  dfContar.head(12)

,Name,Columns,Rows
0,trip_data_12.csv,14,13971118
1,trip_data_11.csv,14,14388451
2,trip_data_10.csv,14,15004556
3,trip_data_9.csv,14,14107693
4,trip_data_8.csv,14,12597109
5,trip_data_7.csv,14,13823840
6,trip_data_6.csv,14,14385456
7,trip_data_5.csv,14,15285049
8,trip_data_4.csv,14,15100468
9,trip_data_3.csv,14,15749228


###4 Identificar si existen renglones con errores en los datos.<br/>
Por ejemplo si hay columnas de mas (o de menos), si hay campos vacios, etc. Si se detectan renglones con errores:<br/>
A)indicar claramente cuantos son los renglones con errores y a que archivos corresponden <br/>
B)crear un nuevo conjunto de datos con los errores eliminados <br/>

In [0]:
files =[plantilla.format(month) for month in range(1,13)]


In [0]:
dfLimpiar = pd.DataFrame(columns=['Name', 'Columns', 'Rows','CantNulos','FueraRangoPasaje','FueraRangoNY','FueraMillas','RowsFinales'])

In [16]:
#Montamos la unidad de google para trabajar con ello y evitar que se llene el espacio de almacenamiento
from google.colab import drive
drive.mount('/content/gdrive' ,force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
quedafiltros=[]

In [0]:
def limpiarDataFrame(df):
    a=len(df.axes[0])
    df=df.dropna()
    #df = df[(df['passenger_count'] >0) & (df['passenger_count'] <= 7)]
    quedafiltros.append(a-len(df.axes[0]))
    a=len(df.axes[0])
    #df=df.dropna()
    df = df[(df['passenger_count'] >0) & (df['passenger_count'] <= 7)]
    quedafiltros.append(a-len(df.axes[0]))
    a=len(df.axes[0])
    df=df[(df['pickup_latitude'] >40.496136) & (df['pickup_latitude'] <40.897877) & (df['pickup_longitude'] >-74.269459)& (df['pickup_longitude'] <-73.690606)]
    quedafiltros.append(a-len(df.axes[0]))
    a=len(df.axes[0])
    df = df[(df['trip_distance'] <=100)]
    quedafiltros.append(a-len(df.axes[0]))
    return df

In [0]:
#Proceso de limpieza de datos se realiza sobre todos los datos del archivo original
for file in files:  
  df=leer_archivo_depurado(file)
  a=limpiarDataFrame(df)
  fname=file
  a.to_csv(fname)  
  gzipCommand="gzip "+fname  
  os.system(gzipCommand)  
  gzipName=fname+".gz"
  os.system( "mv "+gzipName +" \"/content/gdrive/My Drive/Colab Notebooks/"+gzipName+"\"") 
  total_rows=len(df.axes[0])
  total_cols=len(df.axes[1])
  new_row=pd.DataFrame({'Name' : file , 'Columns' : total_cols, 'Rows':total_rows,'CantNulos':quedafiltros[0],'FueraRangoPasaje':quedafiltros[1],'FueraRangoNY':quedafiltros[2],'FueraMillas':quedafiltros[3],'RowsFinales':total_rows-(quedafiltros[0]+quedafiltros[1]+quedafiltros[2]+quedafiltros[3])},index=[0])
  quedafiltros=[]
  dfLimpiar=pd.concat([new_row,dfLimpiar]).reset_index(drop=True)  

  



In [25]:
dfLimpiar

,Name,Columns,Rows,CantNulos,FueraRangoPasaje,FueraRangoNY,FueraMillas,RowsFinales
0,trip_data_12.csv,10,13971118,112,259,233359,0,13737388
1,trip_data_11.csv,10,14388451,754,136,276720,0,14110841
2,trip_data_10.csv,10,15004556,736,104,170721,0,14832995
3,trip_data_9.csv,10,14107693,78,94,152414,0,13955107
4,trip_data_8.csv,10,12597109,45,320,146214,771,12449759
5,trip_data_7.csv,10,13823840,698,109,206778,0,13616255
6,trip_data_6.csv,10,14385456,338,157,258610,0,14126351
7,trip_data_5.csv,10,15285049,39,86,798965,0,14485959
8,trip_data_4.csv,10,15100468,146,85,257812,0,14842425
9,trip_data_3.csv,10,15749228,293,136,282785,0,15466014


###5. Crear un DataFrame usando la libreria pandas y responder lo siguiente:<br/>
A)Indicar el tiempo en segundos, que tarda la libreria pandas en leer un archivo a un  DataFrame<br/>
B)Cual es la huella de memoria del proceso usado para generar el objeto DataFrame <br/>
C)Cuanta memoria del sistema se usa para crear el objeto anterior <br/>
D)Indicar el tiempo que tarda pandas en obtener el promedio de la distancia de viaje (trip_distance) <br/>

###6. Todos los pasos siguientes deben realizarse usando la libreria dask<br/>
A)Indicar el tiempo que tarda en obtener el promedio de la distancia de viaje (trip_distance)<br/>
B)Indicar el tiempo que tarda en obtener el promedio de la duracion de viaje (trip_time_in_secs)<br/>
C)Con las columnas pickup_datetime y dropoff_datetime como objetos temporales, crear una nueva columna en los dataframes que se llame duracion y compararla renglon a renglon con la columna trip_time_in_secs<br/>
D)Definir un viaje_largo como aquellos en los que la duracion del viaje (trip_time_in_secs) sea mayor a 20 minutos.<br/>


###7. Contar el numero de viajes largos en cada archivo

###8. De los viajes largos. <br/>
A) Identificar el numero de taxis diferentes (la columna medallion contiene un numero que identificada a cada uno de los vehiculos). <br/>
B) Que vehiculos son los que mas viajes realizan en cada mes? Son el mismo vehiculo?

3. Responder las siguientes preguntas:
 

*  Cuantas columnas contiene cada archivo de datos descomprimido
*  Cuantos renglones tiene cada archivo




4. Identificar si existen renglones con errores en los datos, por ejemplo si hay columnas de mas (o de menos), si hay campos vacios, etc. Si se detectan renglones con errores:

*   Indicar claramente cuantos son los renglones con errores y a que archivos corresponden
*   Crear un nuevo conjunto de datos con los errores eliminados



5.  Crear un DataFrame usando la libreria pandas y responder lo siguiente:

A)  Indicar el tiempo en segundos, que tarda la libreria pandas en leer un archivo a un  DataFrame <br/>
B)  Cual es la huella de memoria del proceso usado para generar el objeto DataFrame  <br/>
C)   Cuanta memoria del sistema se usa para crear el objeto anterior  <br/>
D)   Indicar el tiempo que tarda pandas en obtener el promedio de la distancia de viaje (trip_distance)


###9.Hacer una grafica temporal del numero total de pasajeros agregados por dias de la semana y despues por horas del dia. Sugerencia: agregar por el correspondiente periodo temporal y graficar. <br/>
 ¿Que pueden observar de las graficas?